## Compact and full LLM szerver for the Chrome Extension

#### The elte-ik-llm-dlite notebook contains the installation and the experiments for the project.
#### This notebook is the practical runnable server.

### (0) Server starting

d:

cd D:\elte-ik-adatbanyaszat

venv\Scripts\activate

jupyter notebook

### (1) Load the pretrained model

Load and prepare the dlite-v2-1_5b LLM to the GPU

In [1]:
import sys
sys.path.insert(1, './dlite-v2-1_5b/')
from instruct_pipeline import InstructionTextGenerationPipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("dlite-v2-1_5b", padding_side="left")
model = AutoModelForCausalLM.from_pretrained("dlite-v2-1_5b", device_map="cuda", torch_dtype=torch.bfloat16)

generate_text = InstructionTextGenerationPipeline(model=model, tokenizer=tokenizer)

### (2) The REST server

Initialise Flask REST CORS server for recive the process requests

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import subprocess
import re

app = Flask(__name__)
CORS(app)

def req_length(text):
    tokens = tokenizer(text)
    tokens_length = len(tokens.attention_mask)
    print('LOG: Input length: ' + str(tokens_length))
    return tokens_length

def req_to_json(title, text):
    return jsonify({title : text})

def req_process(command, text):
    req = command + ' ' + text
    print('LOG: ' + req)
    return generate_text(req)

def req_clean(text):
    cleaned_text = text.replace('\n', ' ').replace('\r', ' ')
    cleaned_text = re.sub(r'[^\w\s.]', '', cleaned_text)
    return cleaned_text

@app.route('/sum', methods=['POST'])
def sum_req():
    data = request.get_json()
    print(data)
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Summarize in one sentence:', text))
        #return req_to_json('response', "TEST TEXT")
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/key', methods=['POST'])
def key_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Give some keywords from this text:', text))
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/title', methods=['POST'])
def title_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    if(req_length(text) <= 1024):
        return req_to_json('response', req_process('Give the most important word in the text:', text))
    else:
        return req_to_json('error', 'Too long text. Limit is 1024 token.')

@app.route('/length', methods=['POST'])
def length_req():
    data = request.get_json()
    text = req_clean(data.get('text', ''))
    ret = {'response': req_length(text)}
    return jsonify(ret)

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/May/2024 13:35:42] "POST /length HTTP/1.1" 200 -


LOG: Input length: 217
LOG: Input length: 217
LOG: Give the most important word in the text: The cat Felis catus commonly referred to as the domestic cat or house cat is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Recent advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a house pet and farm cat but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey like mice and rats. It has a strong flexible body quick reflexes sharp teeth and its night vision and sense of smell are well developed. It is a social species but a solitary hunter and a crepuscular predator. Cat communication includes vocalizations like meowing purring trilling hissing growling and grunting as well as cat body language. It can hear sounds 

127.0.0.1 - - [17/May/2024 13:36:00] "POST /title HTTP/1.1" 200 -


LOG: Input length: 217
LOG: Give some keywords from this text: The cat Felis catus commonly referred to as the domestic cat or house cat is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Recent advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a house pet and farm cat but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey like mice and rats. It has a strong flexible body quick reflexes sharp teeth and its night vision and sense of smell are well developed. It is a social species but a solitary hunter and a crepuscular predator. Cat communication includes vocalizations like meowing purring trilling hissing growling and grunting as well as cat body language. It can hear sounds too faint or too high in frequ

...

127.0.0.1 - - [17/May/2024 13:39:18] "POST /sum HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2024 14:02:16] "POST /length HTTP/1.1" 200 -


LOG: Input length: 217
LOG: Input length: 217
LOG: Give the most important word in the text: The cat Felis catus commonly referred to as the domestic cat or house cat is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Recent advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a house pet and farm cat but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey like mice and rats. It has a strong flexible body quick reflexes sharp teeth and its night vision and sense of smell are well developed. It is a social species but a solitary hunter and a crepuscular predator. Cat communication includes vocalizations like meowing purring trilling hissing growling and grunting as well as cat body language. It can hear sounds 

127.0.0.1 - - [17/May/2024 14:02:43] "POST /title HTTP/1.1" 200 -


LOG: Input length: 217
LOG: Give the most important word in the text: The cat Felis catus commonly referred to as the domestic cat or house cat is a small domesticated carnivorous mammal. It is the only domesticated species of the family Felidae. Recent advances in archaeology and genetics have shown that the domestication of the cat occurred in the Near East around 7500 BC. It is commonly kept as a house pet and farm cat but also ranges freely as a feral cat avoiding human contact. It is valued by humans for companionship and its ability to kill vermin. Its retractable claws are adapted to killing small prey like mice and rats. It has a strong flexible body quick reflexes sharp teeth and its night vision and sense of smell are well developed. It is a social species but a solitary hunter and a crepuscular predator. Cat communication includes vocalizations like meowing purring trilling hissing growling and grunting as well as cat body language. It can hear sounds too faint or too high i

127.0.0.1 - - [17/May/2024 14:38:31] "POST /title HTTP/1.1" 200 -
